In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', 50)

Using TensorFlow backend.


In [67]:
df=pd.read_csv('C:/Users/anand/Desktop/shreyansh sir/ML725/Assignment1/test.csv')
df.head()

,instance (id),date,season,hour,holiday,weekday,working day,weather situation,temperature,atemperature,humidity,wind speed
0,0,2012-05-30,2,18,0,Wednesday,1,1,0.74,0.6667,0.48,0.1642
1,1,2011-06-30,3,3,0,Thursday,1,1,0.60,0.6061,0.60,0.1642
2,2,2011-12-22,1,8,0,Thursday,1,2,0.36,0.3788,0.87,3.0000
3,3,2011-11-13,4,19,0,Sunday,0,1,0.50,0.4848,0.29,0.3582
4,4,2012-01-05,1,7,0,Thursday,1,2,0.20,0.2273,0.69,0.0896


In [69]:
df.shape

(3514, 12)

In [65]:
df['season'].unique()

array([2, 3, 1, 4], dtype=int64)

In [24]:
def out(x):
    if ((x < df['wind speed'].mean() + 2 * df['wind speed'].std()) and (x > df['wind speed'].mean() - 2 * df['wind speed'].std())):
        return 0
    else:
        return 1

In [26]:
df['out'] = df['wind speed'].apply(lambda x: out(x))

In [27]:
df.out.value_counts()

0    12440
1     1425
Name: out, dtype: int64

In [28]:
df.drop(df[df['out'] == 1].index, inplace = True) 

In [29]:
df.shape

(12440, 14)

In [82]:
def binning(i):
    if i in [1,2,3,4]:
        return 1
    elif i in [5,6,7,8]:
        return 2
    elif i in [9, 10, 11, 12]:
        return 3
    elif i in [13,14,15,16]:
        return 4
    elif i in [17,18,19,20]:
        return 5
    elif i in [21,22,23,24]:
        return 6

In [83]:
df['hour_bin']= df.hour.apply(lambda x : binning(x))
df.head()

,instance (id),date,season,hour,holiday,weekday,working day,weather situation,temperature,atemperature,humidity,wind speed,count,out,hour_bin
0,0,2011-09-30,4,5,0,Friday,1,1,0.52,0.5000,0.83,0.1940,26,0,2.0
1,1,2011-06-03,2,11,0,Friday,1,1,0.64,0.6212,0.31,0.3284,192,0,3.0
2,2,2012-03-23,2,10,0,Friday,1,2,0.56,0.5303,0.88,0.1045,275,0,3.0
3,3,2012-12-30,1,18,0,Sunday,0,2,0.24,0.2121,3.00,0.2985,125,0,5.0
4,4,2012-02-07,1,15,0,Tuesday,1,1,0.44,0.4394,0.38,0.1940,147,0,4.0


In [128]:
df.shape

(13865, 13)

In [46]:
df['weather situation'] = pd.Categorical(df['weather situation'], categories=[1,2,3,4])

In [47]:
df['weather situation'].unique()

[1, 2, 3]
Categories (3, int64): [1, 2, 3]

In [48]:
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
df.dtypes

instance (id)                 int64
date                 datetime64[ns]
season                        int64
hour                          int64
holiday                       int64
weekday                      object
working day                   int64
weather situation          category
temperature                 float64
atemperature                float64
humidity                    float64
wind speed                  float64
dtype: object

In [49]:
df.loc[:, 'month'] = df.date.dt.month
df.loc[:, 'week_of_month'] = np.ceil(df.date.dt.day / 7)
df.head()

,instance (id),date,season,hour,holiday,weekday,working day,weather situation,temperature,atemperature,humidity,wind speed,month,week_of_month
0,0,2012-05-30,2,18,0,Wednesday,1,1,0.74,0.6667,0.48,0.1642,5,5.0
1,1,2011-06-30,3,3,0,Thursday,1,1,0.60,0.6061,0.60,0.1642,6,5.0
2,2,2011-12-22,1,8,0,Thursday,1,2,0.36,0.3788,0.87,3.0000,12,4.0
3,3,2011-11-13,4,19,0,Sunday,0,1,0.50,0.4848,0.29,0.3582,11,2.0
4,4,2012-01-05,1,7,0,Thursday,1,2,0.20,0.2273,0.69,0.0896,1,1.0


In [51]:
# dropping unnecessary features for now
df = df.drop(['instance (id)', 'date', 'temperature', 'working day' ], axis=1)
df.head()

,season,hour,holiday,weekday,weather situation,atemperature,humidity,wind speed,month,week_of_month
0,2,18,0,Wednesday,1,0.6667,0.48,0.1642,5,5.0
1,3,3,0,Thursday,1,0.6061,0.60,0.1642,6,5.0
2,1,8,0,Thursday,2,0.3788,0.87,3.0000,12,4.0
3,4,19,0,Sunday,1,0.4848,0.29,0.3582,11,2.0
4,1,7,0,Thursday,2,0.2273,0.69,0.0896,1,1.0


In [52]:
# Creating dummies

df = pd.get_dummies(data=df, columns=['season',  'weekday', 'weather situation', 'hour', 'month', 'week_of_month'], drop_first=True)
df.head()

,holiday,atemperature,humidity,wind speed,season_2,season_3,season_4,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,weather situation_2,weather situation_3,weather situation_4,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,week_of_month_2.0,week_of_month_3.0,week_of_month_4.0,week_of_month_5.0
0,0,0.6667,0.48,0.1642,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,0,0.6061,0.60,0.1642,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,0,0.3788,0.87,3.0000,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,0,0.4848,0.29,0.3582,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
4,0,0.2273,0.69,0.0896,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
# scaling
import pandas as pd
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
df[['wind speed','humidity']] = scaler.fit_transform(df[['wind speed','humidity']])

In [54]:
df.head()

,holiday,atemperature,humidity,wind speed,season_2,season_3,season_4,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,weather situation_2,weather situation_3,weather situation_4,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,week_of_month_2.0,week_of_month_3.0,week_of_month_4.0,week_of_month_5.0
0,0,0.6667,0.160000,0.054733,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,0,0.6061,0.200000,0.054733,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,0,0.3788,0.290000,1.000000,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,0,0.4848,0.096667,0.119400,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
4,0,0.2273,0.230000,0.029867,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
print(df.shape)

(3514, 54)


In [44]:
df['count'].min()

0

In [38]:
print(df.shape)
y = df['count']
X = df.drop('count', axis=1)
print(y.shape)
print(X.shape)

(12440, 55)
(12440,)
(12440, 54)


In [39]:
df.columns

Index(['holiday', 'atemperature', 'humidity', 'wind speed', 'count',
       'season_2', 'season_3', 'season_4', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Thursday',
       'weekday_Tuesday', 'weekday_Wednesday', 'weather situation_2',
       'weather situation_3', 'weather situation_4', 'hour_1', 'hour_2',
       'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9',
       'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15',
       'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21',
       'hour_22', 'hour_23', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12', 'week_of_month_2.0', 'week_of_month_3.0',
       'week_of_month_4.0', 'week_of_month_5.0'],
      dtype='object')

### Modelling

In [40]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [41]:
X.shape[1]

54

In [42]:
X.columns

Index(['holiday', 'atemperature', 'humidity', 'wind speed', 'season_2',
       'season_3', 'season_4', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday', 'weather situation_2', 'weather situation_3',
       'weather situation_4', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'month_10', 'month_11', 'month_12', 'week_of_month_2.0',
       'week_of_month_3.0', 'week_of_month_4.0', 'week_of_month_5.0'],
      dtype='object')

In [43]:
model = Sequential()
model.add(Dense(1, input_dim=X.shape[1], 
                kernel_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=42),
               bias_initializer='zeros'))
opt = SGD(lr=0.05) 
model.compile(loss='mean_squared_error', optimizer=opt)
model.fit(X, y, epochs=20000, batch_size=X.shape[0])

Epoch 1/20000
12440/12440 [==============================] - 0s 14us/step - loss: 67537.0625
Epoch 2/20000
12440/12440 [==============================] - 0s 2us/step - loss: 53687.7617
Epoch 3/20000
12440/12440 [==============================] - 0s 2us/step - loss: 45013.0312
Epoch 4/20000
12440/12440 [==============================] - 0s 2us/step - loss: 39552.1484
Epoch 5/20000
12440/12440 [==============================] - 0s 2us/step - loss: 36087.8086
Epoch 6/20000
12440/12440 [==============================] - 0s 2us/step - loss: 33864.2852
Epoch 7/20000
12440/12440 [==============================] - 0s 2us/step - loss: 32412.3477
Epoch 8/20000
12440/12440 [==============================] - 0s 2us/step - loss: 31440.6484
Epoch 9/20000
12440/12440 [==============================] - 0s 2us/step - loss: 30768.2891
Epoch 10/20000
12440/12440 [==============================] - 0s 2us/step - loss: 30282.9316
Epoch 11/20000
12440/12440 [==============================] - 0s 2us/step - lo

Epoch 90/20000
12440/12440 [==============================] - 0s 2us/step - loss: 20841.1250
Epoch 91/20000
12440/12440 [==============================] - 0s 2us/step - loss: 20772.4844
Epoch 92/20000
12440/12440 [==============================] - 0s 2us/step - loss: 20704.5215
Epoch 93/20000
12440/12440 [==============================] - 0s 2us/step - loss: 20637.2266
Epoch 94/20000
12440/12440 [==============================] - 0s 2us/step - loss: 20570.5977
Epoch 95/20000
12440/12440 [==============================] - 0s 2us/step - loss: 20504.6191
Epoch 96/20000
12440/12440 [==============================] - 0s 1us/step - loss: 20439.2871
Epoch 97/20000
12440/12440 [==============================] - 0s 2us/step - loss: 20374.5898
Epoch 98/20000
12440/12440 [==============================] - 0s 2us/step - loss: 20310.5332
Epoch 99/20000
12440/12440 [==============================] - 0s 2us/step - loss: 20247.0918
Epoch 100/20000
12440/12440 [==============================] - 0s 1us/

Epoch 178/20000
12440/12440 [==============================] - 0s 1us/step - loss: 16723.7051
Epoch 179/20000
12440/12440 [==============================] - 0s 1us/step - loss: 16693.3164
Epoch 180/20000
12440/12440 [==============================] - 0s 1us/step - loss: 16663.1953
Epoch 181/20000
12440/12440 [==============================] - 0s 1us/step - loss: 16633.3398
Epoch 182/20000
12440/12440 [==============================] - 0s 1us/step - loss: 16603.7422
Epoch 183/20000
12440/12440 [==============================] - 0s 1us/step - loss: 16574.4004
Epoch 184/20000
12440/12440 [==============================] - 0s 1us/step - loss: 16545.3184
Epoch 185/20000
12440/12440 [==============================] - 0s 1us/step - loss: 16516.4922
Epoch 186/20000
12440/12440 [==============================] - 0s 1us/step - loss: 16487.9199
Epoch 187/20000
12440/12440 [==============================] - 0s 1us/step - loss: 16459.5957
Epoch 188/20000
12440/12440 [==============================]

12440/12440 [==============================] - 0s 1us/step - loss: 14858.4775
Epoch 266/20000
12440/12440 [==============================] - 0s 2us/step - loss: 14844.0684
Epoch 267/20000
12440/12440 [==============================] - 0s 1us/step - loss: 14829.7852
Epoch 268/20000
12440/12440 [==============================] - 0s 1us/step - loss: 14815.6113
Epoch 269/20000
12440/12440 [==============================] - 0s 2us/step - loss: 14801.5645
Epoch 270/20000
12440/12440 [==============================] - 0s 1us/step - loss: 14787.6338
Epoch 271/20000
12440/12440 [==============================] - 0s 1us/step - loss: 14773.8213
Epoch 272/20000
12440/12440 [==============================] - 0s 1us/step - loss: 14760.1211
Epoch 273/20000
12440/12440 [==============================] - 0s 1us/step - loss: 14746.5312
Epoch 274/20000
12440/12440 [==============================] - 0s 1us/step - loss: 14733.0576
Epoch 275/20000
12440/12440 [==============================] - 0s 1us/step -

Epoch 353/20000
12440/12440 [==============================] - 0s 1us/step - loss: 13954.3975
Epoch 354/20000
12440/12440 [==============================] - 0s 1us/step - loss: 13947.4033
Epoch 355/20000
12440/12440 [==============================] - 0s 1us/step - loss: 13940.4639
Epoch 356/20000
12440/12440 [==============================] - 0s 1us/step - loss: 13933.5850
Epoch 357/20000
12440/12440 [==============================] - 0s 1us/step - loss: 13926.7559
Epoch 358/20000
12440/12440 [==============================] - 0s 1us/step - loss: 13919.9863
Epoch 359/20000
12440/12440 [==============================] - 0s 2us/step - loss: 13913.2686
Epoch 360/20000
12440/12440 [==============================] - 0s 2us/step - loss: 13906.6055
Epoch 361/20000
12440/12440 [==============================] - 0s 1us/step - loss: 13899.9883
Epoch 362/20000
12440/12440 [==============================] - 0s 2us/step - loss: 13893.4346
Epoch 363/20000
12440/12440 [==============================]

...

Epoch 19591/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.5605
Epoch 19592/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5635
Epoch 19593/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.5596
Epoch 19594/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5586
Epoch 19595/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.5596
Epoch 19596/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5625
Epoch 19597/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5586
Epoch 19598/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5596
Epoch 19599/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5586
Epoch 19600/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5586
Epoch 19601/20000
12440/12440 [=========

Epoch 19677/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.5234
Epoch 19678/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.5244
Epoch 19679/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5225
Epoch 19680/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5205
Epoch 19681/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5244
Epoch 19682/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5244
Epoch 19683/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.5205
Epoch 19684/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.5205
Epoch 19685/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.5195
Epoch 19686/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.5195
Epoch 19687/20000
12440/12440 [=========

Epoch 19763/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4883
Epoch 19764/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4873
Epoch 19765/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4883
Epoch 19766/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4863
Epoch 19767/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4873
Epoch 19768/20000
12440/12440 [==============================] - 0s 3us/step - loss: 12428.4854
Epoch 19769/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4834
Epoch 19770/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4863
Epoch 19771/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4834
Epoch 19772/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4834
Epoch 19773/20000
12440/12440 [=========

Epoch 19849/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4531
Epoch 19850/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4502
Epoch 19851/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4502
Epoch 19852/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4531
Epoch 19853/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4502
Epoch 19854/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4492
Epoch 19855/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4492
Epoch 19856/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4502
Epoch 19857/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4502
Epoch 19858/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4473
Epoch 19859/20000
12440/12440 [=========

Epoch 19935/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4170
Epoch 19936/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4170
Epoch 19937/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4170
Epoch 19938/20000
12440/12440 [==============================] - 0s 1us/step - loss: 12428.4141
Epoch 19939/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4141
Epoch 19940/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4150
Epoch 19941/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4141
Epoch 19942/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4170
Epoch 19943/20000
12440/12440 [==============================] - 0s 2us/step - loss: 12428.4141
Epoch 19944/20000
12440/12440 [==============================] - 0s 3us/step - loss: 12428.4170
Epoch 19945/20000
12440/12440 [=========

In [111]:
score = model.evaluate(xtest, ytest, batch_size=X.shape[0])
score

2773/2773 [==============================] - 0s 26us/step


12594.736328125

In [ ]:
16934.1699
16934.1699
16824.3555

In [112]:
score = model.evaluate(xtrain, ytrain, batch_size=X.shape[0])
score

11092/11092 [==============================] - 0s 1us/step


12324.7978515625

### Prediction

In [ ]:
# p = model.predict(xtrain)


In [56]:
prediction = model.predict(df)
print(prediction)

[[490.38452]
 [ 70.88786]
 [286.31982]
 ...
 [327.45538]
 [395.10315]
 [ 76.90593]]


In [57]:
count=0
for i in prediction:
    if i<0:
        count+=1
        print(i)
print(count)

[-76.20943]
[-52.37406]
[-28.175198]
[-88.892845]
[-12.719849]
[-64.663734]
[-36.63196]
[-24.23294]
[-3.452324]
[-74.95018]
[-23.23893]
[-26.029041]
[-109.4389]
[-60.80423]
[-39.73808]
[-113.18258]
[-51.66311]
[-20.97594]
[-63.787712]
[-29.75032]
[-6.143181]
[-22.287685]
[-67.15376]
[-15.743591]
[-62.747135]
[-19.128723]
[-100.59772]
[-59.016014]
[-10.548126]
[-78.85527]
[-129.96527]
[-39.83287]
[-24.498444]
[-4.216484]
[-22.553482]
[-32.86554]
[-67.2413]
[-1.9753036]
[-148.56334]
[-77.53405]
[-23.269516]
[-26.369152]
[-21.425552]
[-7.353195]
[-62.08674]
[-31.906872]
[-9.6325]
[-16.212341]
[-48.19006]
[-7.0272217]
[-26.28186]
[-87.23448]
[-6.6294403]
[-62.690384]
[-34.205124]
[-33.39404]
[-1.3128357]
[-62.592655]
[-92.17802]
[-35.578293]
[-23.98256]
[-11.037735]
[-72.43277]
[-49.32126]
[-81.15104]
[-33.883095]
[-78.507]
[-64.3723]
[-16.581543]
[-2.9635468]
[-84.68776]
[-32.65162]
[-64.17501]
[-2.6874847]
[-33.712738]
[-92.4418]
[-57.82084]
[-102.56978]
[-47.50319]
[-115.84284]
[-35.050

In [58]:
t = pd.DataFrame(prediction, columns=['count']).reset_index(drop=False)
t.columns = ['instance (id)', 'count']
t.head()

,instance (id),count
0,0,490.384521
1,1,70.887863
2,2,286.319824
3,3,332.809814
4,4,124.265305


In [59]:
t['count'] = t['count'].astype('int')
t['count'] = t['count'].apply(lambda x : 0 if x < 0 else x)
t.to_csv('out1.csv', index=False)


In [60]:
t['count'].min()

0